# Ikeda for S175
Scope:
* S175 will be calculated in ScoresII:
   * S175 geometry in a *Seaway input file* will be parsed
   * *Seaway input file* will be converted to ScoresII
   * Draught need to be adjusted to 9.5 m

In [ ]:
# %load ../../imports.py
"""
These is the standard setup for the notebooks.
"""

%matplotlib inline
%load_ext autoreload
%autoreload 2

from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
#plt.style.use('paper')

#import data
import copy
from rolldecay.bis_system import BisSystem
from rolldecay import database
from mdldb.tables import Run

from sklearn.pipeline import Pipeline
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer
from rolldecayestimators.direct_estimator_cubic import EstimatorQuadraticB, EstimatorCubic
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator


In [ ]:
import re

## S175 geometry in a Seaway input file will be parsed

In [ ]:
with open('S175.seaway', mode='r') as file:
    lines = file.readlines()

In [ ]:
lines[0:3]

In [ ]:
regexp_line=re.compile('.*')
def read_line(line):
    result = regexp_line.search(line)
    if not result:
        raise ValueError('Cannot read line:%s'%lins)
    return result.group(0)

def read_line_values(line):
    s = read_line(line)
    regexp_parts = re.compile('(\S+)')
    return regexp_parts.findall(s)
        
    

In [ ]:
read_line(line=lines[0])

In [ ]:
seaway={}
seaway['version'] = read_line(lines[0])
seaway['text'] = read_line(lines[1])

In [ ]:
## Definition of Longitudinal Values:
values = read_line_values(lines[2])
keys=['DR','TR','RLPP','RLA']
for key,value in zip(keys,values):
    seaway[key]=float(value)
seaway

In [ ]:
seaway['NS']=int(read_line(lines[3]))
seaway['NS']

In [ ]:
seaway['DX']=np.array(read_line_values(lines[4]) + 
                      read_line_values(lines[5]) + 
                      read_line_values(lines[6]), dtype=float)
seaway['DX']

In [ ]:
seaway['X']=np.concatenate([[0],np.cumsum(seaway['DX'])])
seaway['X']

In [ ]:
seaway['KCON']=int(read_line(lines[7]))
seaway['KCON']  # KCON = 2: input-sequence is [Z(J,I),Y(J,I)],

In [ ]:
NS=seaway['NS']+1  # one extra?
empty=np.empty(shape=NS)*np.NaN
keys = ['SNR','NWL','SDIST']
for key in keys:
    seaway[key] = empty.copy()

index=8
seaway['Y']=[]
seaway['Z']=[]

for J in range(NS):
    values = read_line_values(lines[index])
    seaway['SNR'][J]=float(values[0])
    seaway['NWL'][J]=NWL=int(float(values[1]))
    seaway['SDIST'][J]=SNR=float(values[2])
    section_values=[]
    while(len(section_values)<(2*(NWL+1))):
        index+=1
        section_values+=read_line_values(lines[index])
    
    Y=np.empty(shape=NWL)*np.NaN
    Z=np.empty(shape=NWL)*np.NaN
    
    for I in range(NWL):
        i=2*I
        Z[I]=section_values[i]
        Y[I]=section_values[i+1]
        
    seaway['Y'].append(np.array(Y, dtype=float))
    seaway['Z'].append(np.array(Z, dtype=float))
    
    index+=1

In [ ]:
def body_plan(Y,Z):
    fig,ax=plt.subplots()
    frames,points = Y.shape
    N=int(np.floor(frames/2))
    
    for y,z in zip(Y[0:N,:], Z[0:N,:]):
        ax.plot(-y,z)
    
    for y,z in zip(Y[N:,:], Z[N:,:]):
        ax.plot(y,z)
    

In [ ]:
N=int(seaway['NWL'].max())*5
X=np.empty(shape=(NS,N))*np.NaN
Y=np.empty(shape=(NS,N))*np.NaN
Z=np.empty(shape=(NS,N))*np.NaN
for i,(y,z) in enumerate(zip(seaway['Y'], seaway['Z'])):
    
    x=np.arange(N)
    xp=np.arange(len(y))
    y_interp=np.interp(x=x, xp=xp, fp=y)
    z_interp=np.interp(x=x, xp=xp, fp=z)
    
    Y[i,:]=y_interp 
    Z[i,:]=z_interp
    X[i,:]=np.ones(shape=N)*seaway['X'][i]

In [ ]:
body_plan(Y,Z)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
from matplotlib import cm

fig = plt.figure()
#fig.set_size_inches(10,10)
ax = fig.gca(projection='3d',)
ax.view_init(45, 20)

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                       linewidth=0, antialiased=True)

ax.set_xlim3d([X.min(), X.max()])
ax.set_ylim3d([Y.min(), Y.max()])
ax.set_zlim3d([Z.min(), Z.max()])

#ax.set_aspect('equal')
#fig.set_size_inches(15,20)


## Draught need to be adjusted to 9.5 m

In [ ]:
T=9.5
N=40
Xs=[]
Ys=[]
Zs=[]

for i,(y,z) in enumerate(zip(seaway['Y'], seaway['Z'])):
        
    mask=y > 0
    zp=z[mask]
    yp=y[mask]
    
    if zp[0] > T:
        continue
    
    z_interp=np.linspace(zp[0],T,N)
    y_interp=np.interp(x=z_interp, xp=zp, fp=yp)
    
    Ys.append(y_interp) 
    Zs.append(z_interp)
    Xs.append(np.ones(shape=N)*seaway['X'][i])
    
Y=np.array(Ys)
Z=np.array(Zs)
X=np.array(Xs)

In [ ]:
body_plan(Y,Z)

## Seaway input file will be converted to ScoresII

In [ ]:
areas=[]
SECOEs=[]
DRAFTs=[]
BSTARs=[]
ZBARs=[]

for y,z in zip(Y, Z):
    area=np.trapz(y=y,x=z)
    areas.append(area)
    
    BSTAR=2*y[-1]
    BSTARs.append(BSTAR)
    
    DRAFT_=z[-1]-z[0]
    DRAFTs.append(DRAFT_)
    
    c=area/(y.max()*DRAFT_)
    SECOEs.append(c)
    
    lever=np.flipud(T-z)
    y_=np.flipud(y)*lever
    
    ZBAR=np.trapz(y=y_,x=lever)/area
    ZBARs.append(ZBAR)

In [ ]:
x=X[:,0]
#x-=x[0]
#x

In [ ]:
from scipy.integrate import simps
2*simps(y=areas,x=x)

In [ ]:
def reduce_sections(xp,fp,lpp, N=21):
    x=np.linspace(0,lpp,N)
    y=np.interp(x=x, xp=xp, fp=fp)
    return x,y

In [ ]:
from rolldecayestimators.tests import test_ikeda_speed_S175 as ti
lpp=ti.L
x_21, areas_21=reduce_sections(x, areas, lpp)
x_21, SECOEs_21=reduce_sections(x,SECOEs, lpp)
x_21, DRAFTs_21=reduce_sections(x,DRAFTs, lpp)
x_21, BSTARs_21=reduce_sections(x,BSTARs, lpp)
x_21, ZBARs_21=reduce_sections(x,ZBARs, lpp)

In [ ]:
lcb=np.trapz(y=areas_21*x_21,x=x_21)/np.trapz(y=areas_21,x=x_21)
displacement=2*np.trapz(y=areas_21,x=x_21)

In [ ]:
lcb

In [ ]:
x_21

In [ ]:
fig,ax=plt.subplots()
ax.plot(x,areas);
ax.set_title('Area curve')
ax.set_xlabel('X [m]')
ax.set_ylabel('Section area $[m^2]$')

fig,ax=plt.subplots()
ax.plot(x,SECOEs);
ax.plot(x_21,SECOEs_21,'--');
ax.set_title('Area coefficients')
ax.set_xlabel('X [m]')
ax.set_ylabel(r'Section coefficient: $\frac{A}{B_s \cdot T_s}$')

fig,ax=plt.subplots()
ax.plot(x,BSTARs);
ax.plot(x_21,BSTARs_21,'--');
ax.set_title('Section water line beam')
ax.set_xlabel('X [m]')
ax.set_ylabel(r'Section water line beam [m]')

fig,ax=plt.subplots()
ax.plot(x,DRAFTs);
ax.plot(x_21,DRAFTs_21,'--');
ax.set_title('Section water draft')
ax.set_xlabel('X [m]')
ax.set_ylabel(r'Section water draft [m]')

fig,ax=plt.subplots()
ax.plot(x,ZBARs);
ax.plot(x_21,ZBARs_21,'--');
ax.set_title('Sectional centeroid')
ax.set_xlabel('X [m]')
ax.set_ylabel(r'Sectional centeroid [m]')
ax.plot([x.min(),x.max()],[T/2,T/2],'r-');

## Save to scoresII indata

In [ ]:
from pyscores2.indata import Indata
from pyscores2.runScores2 import Calculation
from pyscores2.output import OutputFile



In [ ]:
indata = Indata()

In [ ]:
indata.cScores=SECOEs_21
indata.ts=DRAFTs_21
indata.bs=BSTARs_21
indata.zbars=ZBARs_21

kxx=seaway
beam=2*BSTARs_21.max()
indata.lpp=lpp
#indata.displacement=ti.disp
indata.displacement=ti.disp
draught=ti.d
indata.draught=draught
indata.g=ti.g
indata.kxx=0.19*beam  # Guessing to get correct omega0
indata.kyy=0.4*lpp
indata.lcb=lcb-lpp/2
indata.lpp=lpp
indata.projectName='S175'
indata.rho=ti.ra
indata.zcg=ti.vcg-draught
indata.waveFrequenciesMin=0.2
indata.waveFrequenciesMax=0.5
indata.waveFrequenciesIncrement=0.006

indata.runOptions["IE"].set_value(1)

In [ ]:
calculation = Calculation(outDataDirectory='scores2/result')
calculation.run(indata=indata)

In [ ]:
output_file = OutputFile(filePath=calculation.outDataPath)
df = output_file.get_result()

fig,ax=plt.subplots()
for index, group in df.groupby(by=['speed','wave direction'], sort=False):
    group.plot(x='frequencies', y='rollAmplitude', style='o-', label=index, ax=ax)
    
ax.grid(True)
ax.legend();
ax.set_ylabel('Roll');

In [ ]:
df_roll_damping = output_file.get_roll_damping()
df_roll_damping

In [ ]:
T = 20
wE = 2 * np.pi * 1 / T  # circular frequency
wE